In [2]:
# Взяты у Yahoo через функцию get_top_sp500_tickers, функция работает долго, так что я создал список
top_20_symbols = ['AAPL','MSFT','NVDA','AMZN','GOOGL','GOOG','META','BRK-B','AVGO',
                  'TSLA','WMT','LLY','JPM','V','UNH','MA','XOM','COST','NFLX','PG']

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import time
#from prophet import Prophet

def get_top_sp500_tickers(top_n=20):
    """Альтернативная версия с получением текущей капитализации через yfinance"""
    try:
        # Получаем базовый список с Wikipedia
        url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
        tables = pd.read_html(url)
        sp500_table = tables[0]
        tickers = sp500_table['Symbol'].tolist()
        tickers = [ticker.replace('.', '-') for ticker in tickers]
        
        # Получаем текущую капитализацию для каждого тикера
        market_caps = {}
        for ticker in tickers:
            try:
                stock = yf.Ticker(ticker)
                market_cap = stock.info.get('marketCap', 0)
                market_caps[ticker] = market_cap
                time.sleep(0.1)  # Чтобы не получить бан от Yahoo
            except:
                continue
        
        # Сортируем по капитализации и берем топ N
        sorted_tickers = sorted(market_caps.keys(), key=lambda x: market_caps[x], reverse=True)
        return sorted_tickers[:top_n]
        
    except Exception as e:
        print(f"Ошибка: {e}")
        return None

def download_and_transform_data(tickers, period="10y"):
    """Загружает данные и преобразует в нужный формат"""
    try:
        data = yf.download(
            tickers=tickers,
            period=period,
            interval="1d",
            group_by='ticker',
            progress=True,
            threads=True
        )
        
        # Собираем данные в список DataFrame'ов
        frames = []
        for ticker in tickers:
            if ticker in data:
                # Создаем копию данных для текущего тикера
                df = data[ticker].copy()
                # Сбрасываем индекс, чтобы 'Date' стал обычным столбцом
                df = df.reset_index()
                # Добавляем столбец с тикером
                df['symbol'] = ticker
                frames.append(df)
        
        if not frames:
            return pd.DataFrame()
            
        # Объединяем все DataFrame'ы
        combined = pd.concat(frames, ignore_index=True)
        
        # Сортируем по тикеру и дате
        combined = combined.sort_values(['symbol', 'Date'])
        
        # Создаем целевую переменную (годовая доходность)
        combined['future_price'] = combined.groupby('symbol')['Close'].shift(-252)
        combined['annual_return'] = (combined['future_price'] / combined['Close']) - 1
        
        return combined
    
    except Exception as e:
        print(f"Ошибка при загрузке данных: {e}")
        return pd.DataFrame()
    
    except Exception as e:
        print(f"Ошибка при загрузке и преобразовании данных: {e}")
        return pd.DataFrame()

def save_to_csv(data, filename_prefix="sp500_ml_data"):
    """Сохраняет данные в CSV файл"""
    if data is None or data.empty:
        print("Нет данных для сохранения")
        return
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{filename_prefix}_{timestamp}.csv"
    data.to_csv(filename, index=False)
    print(f"Данные сохранены в файл: {filename}")

def main(top_tickers=top_20_symbols):
    print("Получаем топ 20 акций S&P 500 по капитализации...")
    if top_tickers == []:
        top_tickers = get_top_sp500_tickers(top_n=20)
    
    if not top_tickers:
        print("Не удалось получить список акций. Завершение работы.")
        return
    
    print(f"Топ 20 акций по капитализации: {', '.join(top_tickers)}")
    
    print("Загружаем и преобразуем исторические данные...")
    stock_data = download_and_transform_data(top_tickers, period="10y")
    
    if not stock_data.empty:
        print("\nУспешно загружены и преобразованы данные:")
        print(stock_data.head())
        
        save_to_csv(stock_data, "sp500_ml_ready")
    else:
        print("Не удалось загрузить данные или данные пусты.")

In [28]:
main()

Получаем топ 20 акций S&P 500 по капитализации...
Топ 20 акций по капитализации: AAPL, MSFT, NVDA, AMZN, GOOGL, GOOG, META, BRK-B, AVGO, TSLA, WMT, LLY, JPM, V, UNH, MA, XOM, COST, NFLX, PG
Загружаем и преобразуем исторические данные...


[*********************100%***********************]  20 of 20 completed



Успешно загружены и преобразованы данные:
Price       Date       Open       High        Low      Close     Volume  \
0     2015-04-13  28.668572  28.713240  28.275517  28.329115  145460400   
1     2015-04-14  28.362616  28.427381  28.119190  28.206287  102098400   
2     2015-04-15  28.230856  28.391650  28.141525  28.313486  115881600   
3     2015-04-16  28.201820  28.384949  28.163855  28.177254  113476000   
4     2015-04-17  28.038793  28.170556  27.795365  27.860130  207828000   

Price symbol  future_price  annual_return  
0       AAPL     25.121523      -0.113226  
1       AAPL     25.485466      -0.096462  
2       AAPL     25.499111      -0.099401  
3       AAPL     24.987312      -0.113210  
4       AAPL     24.448217      -0.122466  
Данные сохранены в файл: sp500_ml_ready_20250412_174743.csv


Target - доходность через год от текущей даты

In [4]:
import pandas as pd
df = pd.read_csv("sp500_ml_ready_20250412_174743.csv")

In [20]:
import pandas as pd
import pandas_ta as ta

# Функция для добавления индикаторов к каждой группе (акции)
def add_ta_indicators(group):
    # Убедимся, что данные отсортированы по дате
    group = group.sort_values('Date')
    
    # Добавляем индикаторы (примеры)
    group.ta.sma(length=50, append=True, col_names=('SMA_50',))  # SMA за 50 дней
    group.ta.ema(length=200, append=True, col_names=('EMA_200',))  # EMA за 200 дней
    group.ta.rsi(length=30, append=True, col_names=('RSI_14',))  # RSI за 14 дней
    group.ta.macd(append=True, col_names=('MACD_', 'MACD_hist_', 'MACD_signal_'))  # MACD
    group.ta.adx(length=50, append=True)  # ADX за 50 дней
    group.ta.bbands(length=20, append=True)  # Bollinger Bands
    group.ta.atr(length=14, append=True)  # ATR (волатильность)
    
    return group

# Применяем функцию к каждой группе (акции)
df_with_ta = df.groupby('symbol').apply(add_ta_indicators)

# Удаляем NaN (если индикаторы их создали)
df_with_ta = df_with_ta.reset_index(drop=True)\
            .dropna()
df_with_ta

df_with_ta.Date = pd.to_datetime(df_with_ta.Date)

C:\Users\user\AppData\Local\Temp\ipykernel_11272\3566150219.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_with_ta = df.groupby('symbol').apply(add_ta_indicators)


In [21]:
df_with_ta.to_csv("stocks_with_indicators_20.csv", 
                  index=0)

import swifter

df_with_ta = df.groupby('symbol').swifter.apply(add_ta_indicators)